In [106]:
!pip3 install -r requirements.txt

You should consider upgrading via the 'python -m pip install --upgrade pip' command.

  Using cached https://files.pythonhosted.org/packages/d6/26/eb28e975b7a37aad38d7ec4f7a0f652bdee6ecf36e6bd06f473c5af9b87b/openpyxl-2.6.4.tar.gz
  Using cached https://files.pythonhosted.org/packages/f0/da/572cbc0bc582390480bbd7c4e93d14dc46079778ed915b505dc494b37c57/jdcal-1.4.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/28/a99c42aea746e18382ad9fb36f64c1c1f04216f41797f2f0fa567da11388/et_xmlfile-1.0.1.tar.gz
  Running setup.py install for et-xmlfile: started
    Running setup.py install for et-xmlfile: finished with status 'done'
  Running setup.py install for openpyxl: started
    Running setup.py install for openpyxl: finished with status 'done'


In [8]:
!{sys.executable} -m spacy download nl_core_news_sm

You should consider upgrading via the 'python -m pip install --upgrade pip' command.

  Running setup.py install for nl-core-news-sm: started
    Running setup.py install for nl-core-news-sm: finished with status 'done'
✔ Download and installation successful
You can now load the model via spacy.load('nl_core_news_sm')


In [2]:
import pandas as pd
from collections import Counter
import numpy as np
import spacy
import nl_core_news_sm
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
import re
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.metrics import roc_auc_score
import xgboost as xgb
import xlrd
#from imblearn.over_sampling import SMOTE
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import openpyxl
import joblib
import pickle

In [3]:
#read in data 
data_1= pd.read_excel("./Heijmans data final.xlsx")
#data_1.head()

In [4]:
#re-naming columns
data_1= data_1.rename(columns ={'Nr.': 'ID_num', "Beschrijving incident": "Incident_Description", "Subcategorie ongeval": "Sub_Categories", "Categorie ongeval": "Main_Categories", "Subcategorie(2) ongeval": "Reason_Categories", 
                                "Datum incident": "Incident_Date" })


#selecting the columns to work with
ID_num = data_1["ID_num"]
Incident_Description = data_1["Incident_Description"]
Sub_Categories = data_1["Sub_Categories"]
Reason_Categories= data_1["Reason_Categories"]
Main_Categories= data_1["Main_Categories"]
Incident_Date= data_1["Incident_Date" ]

#creating a new dataframe
c1= pd.Series(ID_num, name = "ID_num")
c2= pd.Series(Incident_Description, name = "Incident_Description")
c3= pd.Series(Sub_Categories, name= "Sub_Categories")
c4= pd.Series(Reason_Categories ,name= "Reason_Categories")
c5= pd.Series(Main_Categories, name= "Main_Categories")
c6= pd.Series(Incident_Date, name= "Incident_Date")

df = pd.concat([c1,c2,c3,c4,c5,c6], axis=1)
print(df.shape)

# EDA- Exploratory Data Analysis

#number of entries for each column
column_count= df.count()
print ("Column Count")
print(column_count)
print("")
print("")

#number of null/NAN entries 
print("Null Count ") 
print(df.isnull().sum())
print("")
print("")

#Count of uniques entries
a= df["Main_Categories"].nunique()
print("Unique_main_categories:", a)
c= df["Sub_Categories"].nunique()
print("Unique_subcategories:", c)
e= df["Reason_Categories"].nunique()
print("Unique_reasons:", e)
print("")
print("")

# class distribution Percentages 
class_dist= (df.groupby("Main_Categories").size()/df["Main_Categories"].count())*100
print(class_dist)

# removing nan
df["Main_Categories"].replace(r'^\s+$', np.nan, regex=True)
mc_df_1=df.dropna(subset=["Main_Categories"])

print("Count of new data frame ")
print(mc_df_1.count())
print("")
print("")
print("Double Checking for NaN")
print(mc_df_1.isnull().sum())

(18516, 6)
Column Count
ID_num                  18516
Incident_Description    17909
Sub_Categories           3276
Reason_Categories        2340
Main_Categories          3307
Incident_Date           18516
dtype: int64


Null Count 
ID_num                      0
Incident_Description      607
Sub_Categories          15240
Reason_Categories       16176
Main_Categories         15209
Incident_Date               0
dtype: int64


Unique_main_categories: 10
Unique_subcategories: 43
Unique_reasons: 154


Main_Categories
Aanrijding                                      5.624433
Agressie en extreme lichamelijke inspanning     2.056244
Andere vormen van contact met voorwerpen       14.454188
BHV inzet op locatie                            3.386755
Gevaarlijke atmosfeer                           0.302389
Gevaarlijke stoffen, brand en explosies         3.144844
Gezondheidsschade                               0.514061
Machines en handgereedschap                    12.760810
Vallen (van personen of voor

In [5]:
# removing noise from the data 
mc_df_1=mc_df_1[Main_Categories!="overig"]  # other category adds no value to data and reduces model's performance
mc_df_2= mc_df_1[Main_Categories!="Gevaarlijke atmosfeer"] # too few entries, the machine learns nothing
mc_df= mc_df_2[Main_Categories!="Gezondheidsschade"] # too few entries, the machine learns nothing

In [10]:
# class distribution Percentages to establish baseline
class_dist= (mc_df.groupby("Main_Categories").size()/mc_df["Main_Categories"].count())*100
print(class_dist)

Main_Categories
Aanrijding                                      6.467316
Agressie en extreme lichamelijke inspanning     2.364395
Andere vormen van contact met voorwerpen       16.620306
BHV inzet op locatie                            3.894298
Gevaarlijke stoffen, brand en explosies         3.616134
Machines en handgereedschap                    14.673157
Vallen (van personen of voorwerpen)            52.364395
dtype: float64


In [6]:
# If stopwords load fails, load stopwords here
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\micks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
stopword_list = set(nltk.corpus.stopwords.words('dutch'))
#add words that aren't in the NLTK stopwords list
added_stopwords = ["b", "u", "e", "ste", "kv", "aa", "a9", "mtr", '+', '1e', '2e', '<', '>', 'a', 'a1', 'a12', 'a2', 'a9',"één", "gis", "app", "h", 'o.a.', 'oa', 'en/of', "uur", "km", "mm", "v", "kg", "cm", "mm²", "mm2", "0ngever", "e", "de", "m"]
updated_stopword_list = stopword_list.union(added_stopwords)
#print(updated_stopword_list)

# Dutch spacy tokens 
nlp = nl_core_news_sm.load()

def custom_lemmatizer(data):
    return [(token.lemma_) for token in nlp(data) if not token.is_digit and not token.is_punct and not token.is_space]

In [14]:
print(updated_stopword_list)

{'die', 'haar', 'voor', 'veel', 'km', 'kan', 'zo', 'iets', 'tot', 'om', 'ja', 'nu', 'over', 'van', 'in', 'mtr', 'is', 'men', '<', 'uw', 'dat', 'tegen', 'als', 'was', '1e', 'a12', 'één', 'der', '2e', 'gis', 'oa', 'onder', 'wat', 'uit', 'omdat', 'kv', 'o.a.', 'a9', 'op', 'zonder', 'ge', 'mm²', 'er', 'alles', 'deze', 'een', 'mm', 'naar', 'meer', 'je', 'waren', 'de', 'wil', 'hoe', 'door', 'hem', 'h', 'aan', 'nog', 'geweest', 'zou', 'dan', 'al', 'zelf', 'kon', '>', 'na', 'had', 'maar', 'heb', 'cm', 'ben', 'mm2', 'en/of', 'zal', '+', 'hebben', 'ik', 'a', 'wordt', 'a2', 'kg', 'wezen', 'want', 'm', 'het', 'mijn', 'hij', 'kunnen', 'me', 'doen', 'heeft', 'wie', 'ons', 'werd', 'reeds', 'niet', 'uur', 'a1', 'met', 'bij', 'zijn', 'iemand', 'ze', 'zich', 'geen', 'altijd', 'u', 'doch', 'andere', 'e', 'moet', 'en', 'niets', 'te', 'v', 'toch', 'mij', 'dus', 'ste', '0ngever', 'b', 'eens', 'dit', 'zij', 'daar', 'worden', 'hun', 'ook', 'toen', 'of', 'app', 'aa', 'hier'}


In [58]:
# split the dataset into training and validation datasets 
text = mc_df["Incident_Description"].astype(str)
label = mc_df["Main_Categories"]
x_text_train, text_test, label_train, label_test, = model_selection.train_test_split(text,label, test_size = 0.25)

X_train, X_test, y_train, y_test 
    = train_test_split(X, y, test_size=0.2, random_state=1)

 X_train, X_val, y_train, y_val 
    = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


In [9]:
## 𝑇𝐹×𝐼𝐷𝐹
# word level tf-idf
tfidf_vectorizer= TfidfVectorizer(max_features=12000,  stop_words=updated_stopword_list, max_df= 0.5, tokenizer=custom_lemmatizer)


In [60]:
#fit & transform for cross validation 
train_tfidf = tfidf_vectorizer.fit(text)
tfidf_text_v= train_tfidf.transform(text)

# fit and transform for test  & train
tfidf_text_v_train= train_tfidf.transform(text_train)
tfidf_text_v_test= train_tfidf.transform(text_test)

C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['0ngev', 'moeten', 'sen', 'wijzen', 'willen', 'zullen'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [61]:
#basic XGB model
xgb_cl=xgb.XGBClassifier(random_state=4)
xgb_cl.fit(tfidf_text_v_train, label_train)
xgb_cl.score(tfidf_text_v_test, label_test)
print("Accuracy score (test):",xgb_cl.score(tfidf_text_v_test, label_test))
print("Accuracy score (training):", xgb_cl.score(tfidf_text_v_train, label_train))

Accuracy score (test): 0.6648122392211405
Accuracy score (training): 0.9281409364858599


In [35]:
#hyperparameter tuning- XGB model with early stopping and balanced weights and macros as metrics
xgb_cl=xgb.XGBClassifier(random_state=4, class_weight="balanced", metrics='f1', early_stopping_rounds=50)
xgb_cl.fit(tfidf_text_v_train, label_train)
xgb_cl.score(tfidf_text_v_test, label_test)
print("Accuracy score (test):",xgb_cl.score(tfidf_text_v_test, label_test))
print("Accuracy score (training):", xgb_cl.score(tfidf_text_v_train, label_train))


Accuracy score (test): 0.6717663421418637
Accuracy score (training): 0.9304589707927677


In [13]:
labels_predict_xgb= xgb_cl.predict(tfidf_text_v_test)
print(classification_report(label_test, labels_predict_xgb))

                                             precision    recall  f1-score   support

                                 Aanrijding       0.76      0.50      0.60        44
Agressie en extreme lichamelijke inspanning       0.00      0.00      0.00        14
   Andere vormen van contact met voorwerpen       0.59      0.37      0.46       126
                       BHV inzet op locatie       0.79      0.52      0.62        29
    Gevaarlijke stoffen, brand en explosies       0.18      0.07      0.10        30
                Machines en handgereedschap       0.56      0.52      0.54       102
        Vallen (van personen of voorwerpen)       0.72      0.92      0.81       374

                                   accuracy                           0.67       719
                                  macro avg       0.51      0.41      0.45       719
                               weighted avg       0.64      0.67      0.64       719



In [14]:
print(confusion_matrix(label_test, labels_predict_xgb))

[[ 22   0   1   0   1   0  20]
 [  0   0   0   0   0   1  13]
 [  1   1  47   0   3  25  49]
 [  1   0   0  15   0   2  11]
 [  0   0   4   0   2   4  20]
 [  2   5  17   0   4  53  21]
 [  3   2  10   4   1  10 344]]


In [36]:
#Feature/ Noise reduction with min_df
tfidf_vectorizer_2= TfidfVectorizer(max_features= 8000,  stop_words=updated_stopword_list, max_df= 0.5, min_df=0.05, tokenizer=custom_lemmatizer)
#fit & transform for cross validation 
train_tfidf2 =tfidf_vectorizer_2.fit(text)
tfidf_text_v2= train_tfidf2.transform(text)

# fit and transform for test  & train
tfidf_text_v_train2= train_tfidf2.transform(text_train)
tfidf_text_v_test2= train_tfidf2.transform(text_test)



C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['0ngev', 'moeten', 'sen', 'wijzen', 'willen', 'zullen'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [37]:

xgb_cl=xgb.XGBClassifier(random_state=4, class_weight="balanced", metrics='mac', early_stopping_rounds=10)
xgb_cl.fit(tfidf_text_v_train2, label_train)
xgb_cl.score(tfidf_text_v_test2, label_test)
print("Accuracy score (test):",xgb_cl.score(tfidf_text_v_test2, label_test))
print("Accuracy score (training):", xgb_cl.score(tfidf_text_v_train2, label_train))

Accuracy score (test): 0.521557719054242
Accuracy score (training): 0.6226240148354195


In [38]:
labels_predict_xgb= xgb_cl.predict(tfidf_text_v_test2)
print(classification_report(label_test, labels_predict_xgb))

                                             precision    recall  f1-score   support

                                 Aanrijding       0.00      0.00      0.00        44
Agressie en extreme lichamelijke inspanning       0.00      0.00      0.00        14
   Andere vormen van contact met voorwerpen       0.31      0.10      0.15       126
                       BHV inzet op locatie       0.00      0.00      0.00        29
    Gevaarlijke stoffen, brand en explosies       0.00      0.00      0.00        30
                Machines en handgereedschap       0.41      0.22      0.28       102
        Vallen (van personen of voorwerpen)       0.56      0.91      0.69       374

                                   accuracy                           0.52       719
                                  macro avg       0.18      0.17      0.16       719
                               weighted avg       0.40      0.52      0.43       719



C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
print(confusion_matrix(label_test, labels_predict_xgb))

[[  0   0   0   0   0   3  41]
 [  0   0   1   0   0   0  13]
 [  2   1  12   0   0  12  99]
 [  0   0   0   0   0   0  29]
 [  1   0   3   0   0   4  22]
 [  3   0  11   0   1  22  65]
 [  2   4  12   0   2  13 341]]


In [20]:
#Hyperparameter turning with Gridsearch
model_params= {"svm": {"model":svm.SVC(gamma="auto"), "params": {"C": [1,10,20], "kernel":["rbf", "linear"], "penalty":["l1", "L2"], "dual": ["False", "True"], "loss":["squared_hinge"], "class_weight":[0.05, 0.95, 1, 20] }}, 
              "decision_tree": {"model":DecisionTreeClassifier(),"params": {"n_estimators": [1,10], "criterion":['gini','entropy'], "class_weight":[0.05, 0.95, 1, 20]}}, 
               "naive_bayes":{"model":MultinomialNB(),"params": {"C": [1,10],"class_weight":[0.05, 0.95, 1, 20] }},
               "kNN":{"model":KNeighborsClassifier(),"params": { "n_neighbors":[3,5,10], "weights":["uniform", "distance"], "algorithm":['ball_tree','kd_tree'],"class_weight":[0.05, 0.95, 1, 20] }},
               "xgb_cl":{"model":xgb.XGBClassifier(random_state=4),"params": { "n_estimators": [50,100], "learning_rate":[0.1,0.5,1.0],"class_weight":[0.05, 0.95, 1, 20] }}}

In [ ]:
scores=[]

for model_name, mp in model_params.items():
    clf= GridSearchCV(mp["model"], mp ["params"], cv=5, return_train_score=False )
clf.fit(tfidf_text_v_train, label_train)
scores.append({"model": model_name, "best_score":clf.best_score_, "best_params": clf.best_params_})

In [ ]:
print(scores)

In [27]:
#Gridsearch parameter turning sepcifically for XGB model
model_params= {"xgb_cl":{"model":xgb.XGBClassifier(random_state=2),"params": { "n_estimators": [100,150], 
                                                                             "learning_rate":[0.5,1.0],
                                                                             "class_weight":[0.05, 0.10, 0.15, 5],
                                                                             "max_depth":[3,5,7],"min_child_weight":[1,1.5],
                                                                             "scale_pos_weight":[0.5, 0.7, 1,1.5]}}}

In [28]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
scores=[]

for model_name, mp in model_params.items():
    clf= GridSearchCV(mp["model"], mp ["params"], cv=cv, return_train_score=False )
clf.fit(tfidf_text_v_train, label_train)
scores.append({"model": model_name, "best_score":clf.best_score_, "best_params": clf.best_params_})

In [29]:
print(scores)

[{'model': 'xgb_cl', 'best_score': 0.6882315029646817, 'best_params': {'class_weight': 0.05, 'learning_rate': 0.5, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'scale_pos_weight': 0.5}}]


In [30]:
#Getting the test & accuracy scores
xgb_cl_op=xgb.XGBClassifier(random_state=4, learning_rate= 0.5, max_depth= 3,
                            min_child_weight= 1, 
                            n_estimators= 100,
                            class_weight=0.05, 
                            metrics='f1', 
                            early_stopping_rounds=50, 
                            scale_pos_weight=0.5)
xgb_cl_op.fit(tfidf_text_v_train, label_train)
xgb_cl_op.score(tfidf_text_v_test, label_test)
print("Accuracy score (test):",xgb_cl_op.score(tfidf_text_v_test, label_test))
print("Accuracy score (training):", xgb_cl_op.score(tfidf_text_v_train, label_train))


Accuracy score (test): 0.6787204450625869
Accuracy score (training): 0.9077422345850719


In [31]:
labels_predict_xgb= xgb_cl_op.predict(tfidf_text_v_test)
print(classification_report(label_test, labels_predict_xgb))

                                             precision    recall  f1-score   support

                                 Aanrijding       0.82      0.52      0.64        44
Agressie en extreme lichamelijke inspanning       0.08      0.07      0.08        14
   Andere vormen van contact met voorwerpen       0.63      0.37      0.47       126
                       BHV inzet op locatie       0.82      0.48      0.61        29
    Gevaarlijke stoffen, brand en explosies       0.09      0.03      0.05        30
                Machines en handgereedschap       0.55      0.54      0.54       102
        Vallen (van personen of voorwerpen)       0.73      0.93      0.82       374

                                   accuracy                           0.68       719
                                  macro avg       0.53      0.42      0.46       719
                               weighted avg       0.66      0.68      0.65       719



In [32]:
print(confusion_matrix(label_test, labels_predict_xgb))

[[ 23   0   1   0   0   0  20]
 [  0   1   0   0   0   1  12]
 [  1   2  47   0   5  25  46]
 [  1   0   0  14   0   2  12]
 [  0   1   4   0   1   6  18]
 [  1   6  15   0   4  55  21]
 [  2   2   8   3   1  11 347]]


In [78]:
#Minimize features to reduce overfitting via max_def & min_def
# increase number of stop words
tfidf_vectorizer= TfidfVectorizer(max_features=14000,  stop_words=updated_stopword_list, max_df= 0.5, min_df=0.08, tokenizer=custom_lemmatizer)

#fit & transform for cross validation 
train_tfidf = tfidf_vectorizer.fit(text)
tfidf_text_v= train_tfidf.transform(text)

# fit and transform for test  & train
tfidf_text_v_train= train_tfidf.transform(text_train)
tfidf_text_v_test= train_tfidf.transform(text_test)

C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['0ngev', 'moeten', 'sen', 'wijzen', 'willen', 'zullen'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [79]:

xgb_cl_op=xgb.XGBClassifier(random_state=4, learning_rate= 0.5, max_depth= 3,
                            min_child_weight= 1, 
                            n_estimators= 100,
                            class_weight=0.05, 
                            metrics='f1', 
                            early_stopping_rounds=50, 
                            scale_pos_weight=1)
xgb_cl_op.fit(tfidf_text_v_train, label_train)
xgb_cl_op.score(tfidf_text_v_test, label_test)
print("Accuracy score (test):",xgb_cl_op.score(tfidf_text_v_test, label_test))
print("Accuracy score (training):", xgb_cl_op.score(tfidf_text_v_train, label_train))

Accuracy score (test): 0.49235048678720444
Accuracy score (training): 0.5605006954102921


In [80]:
labels_predict_xgb= xgb_cl_op.predict(tfidf_text_v_test)
print(classification_report(label_test, labels_predict_xgb))
print(confusion_matrix(label_test, labels_predict_xgb))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        46
           1       0.00      0.00      0.00        20
           2       0.33      0.01      0.01       139
           3       0.00      0.00      0.00        24
           4       0.00      0.00      0.00        30
           5       0.38      0.21      0.27       115
           6       0.50      0.95      0.66       345

    accuracy                           0.49       719
   macro avg       0.17      0.17      0.13       719
weighted avg       0.37      0.49      0.36       719

[[  0   0   0   0   0   3  43]
 [  0   0   0   0   0   2  18]
 [  0   0   1   0   0  17 121]
 [  0   0   0   0   0   0  24]
 [  0   0   0   0   0   4  26]
 [  0   0   0   0   0  24  91]
 [  0   0   2   0   0  14 329]]


C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
#Undersample the majority class
from sklearn.feature_selection import SelectKBest
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
from sklearn.feature_selection import chi2

tfidf_vectorizer_2= TfidfVectorizer(max_features= 12000,  stop_words=updated_stopword_list, max_df= 0.5, tokenizer=custom_lemmatizer)
#fit & transform for cross validation 
train_tfidf2 =tfidf_vectorizer_2.fit(text)
tfidf_text_v2= train_tfidf2.transform(text)

# fit and transform for test  & train
tfidf_text_v_train2= train_tfidf2.transform(text_train)
tfidf_text_v_test2= train_tfidf2.transform(text_test)

fs = SelectKBest(chi2, k="all")

fs.fit(tfidf_text_v_train, text_train)

X_train_fs = fs.transform(tfidf_text_v_train)

X_test_fs = fs.transform(tfidf_text_v_test)


text_train_rus, label_train_rus = rus.fit_resample(X_train_fs, label_train)

text_test_rus, label_test_rus = rus.fit_resample(X_test_fs, label_test)


C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['0ngev', 'moeten', 'sen', 'wijzen', 'willen', 'zullen'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [49]:
xgb_cl=xgb.XGBClassifier(random_state=4, learning_rate= 0.5, max_depth= 3, min_child_weight= 1, n_estimators= 100, scale_pos_weight=1, class_weight=0.05, metrics='mac', early_stopping_rounds=50)
xgb_cl.fit(text_train_rus, label_train_rus)
xgb_cl.score(X_test_fs , label_test)

0.15577190542420027

USING PAST YEARS TO PREDICTION FUTURE YEARS' CATEGORIES

In [10]:
#Divide dataset 90:10 
mc_df["Incident_Date"] = pd.to_datetime(mc_df["Incident_Date"], format='%Y%m%d')
mc_df = mc_df.set_index("Incident_Date")
mc_df = mc_df.sort_index()
df_ey = mc_df['2006-09-11':'2017-04-10']
df_ly  = mc_df['2017-04-11':]
print('Train Dataset:',df_ey.shape)
print('Test Dataset:',df_ly.shape)


Train Dataset: (984, 5)
Test Dataset: (1892, 5)


In [9]:
text_1 = df_ey["Incident_Description"].astype(str)
text_1=text_1.to_numpy()
label_1 = df_ey["Main_Categories"]
text_1_train, text_1_test, label_1_train, label_1_test, = model_selection.train_test_split(text_1,label_1, test_size = 0.25)

In [11]:
import joblib
import pickle
train_tfidf = tfidf_vectorizer.fit(text_1)
tfidf_text_v= train_tfidf.transform(text_1)
# joblib.dump(train_tfidf, "tfidf_vectorizer.sav")
pickle.dump(train_tfidf, open('tfidf_vectorizer.pickle', 'wb'))
# fit and transform for test  & train
tfidf_text_v_train= train_tfidf.transform(text_1_train)
tfidf_text_v_test= train_tfidf.transform(text_1_test)


In [19]:

model_name = "Category Ongeval(pototype).sav"
svm = LinearSVC(penalty="l1", loss='squared_hinge',dual=False )
SVM_cl_p = CalibratedClassifierCV(svm)
SVM_cl_p.fit(tfidf_text_v_train, label_1_train)
SVM_cl_p.fit(tfidf_text_v_test, label_1_test)
print("SVM_P score (test):",SVM_cl_p.score(tfidf_text_v_test, label_1_test))
print("SVM_P  score (training):", SVM_cl_p.score(tfidf_text_v_train, label_1_train))

joblib.dump(SVM_cl_p, model_name)
#print ("SVM_P Score:", SVM_cl_p.score(tfidf_text_v_test, label_test))



SVM_P score (test): 0.8373983739837398
SVM_P  score (training): 0.5989159891598916


['Category Ongeval(pototype).sav']

## Train, Test, Validation split with cross validation on test

### Splitting up the data

Split:
- CV Train (80%):
    - Train (50%):
        - Train_1 (50%)
        - Train_2 (50%)
    - Train (50%):
        - Train_3 (50%)
        - Train_4 (50%)
- Test (20%)

##### Variables to be used :-
- Test Data:
    - x_test_set
    - y_test_set
- Train Data:
    - x_train_1, y_train_1
    - x_train_2, y_train_2
    - x_train_3, y_train_3
    - x_train_4, y_train_4

In [46]:
features = df_ey["Incident_Description"].astype(str)
feature_vectorizer = tfidf_vectorizer.fit(features)
labels = df_ey["Main_Categories"]
# Splitting data into train and test data (80/20)
X_train, x_test_set, Y_train, y_test_set, = model_selection.train_test_split(features, labels, stratify=labels, test_size = 0.2, random_state=3)

# Splitting train data into train and validation data 4 ways (25/25/25/25)
pre_x_train, pre_x_cv, pre_y_train, pre_y_cv, = model_selection.train_test_split(X_train, Y_train, stratify=Y_train, test_size = 0.5, random_state=3)

x_train_1, x_train_2, y_train_1, y_train_2 = model_selection.train_test_split(pre_x_train, pre_y_train, stratify=pre_y_train, test_size = 0.5, random_state=3)
x_train_3, x_train_4, y_train_3, y_train_4 = model_selection.train_test_split(pre_x_cv, pre_y_cv, stratify=pre_y_cv, test_size = 0.5, random_state=3)

x_test_transformed = feature_vectorizer.transform(x_test_set)


#### Just checking the shape of the data

In [47]:
print("Testing features dataset shape: ", x_test_set.shape)
print("Testing labels dataset shape: ", y_test_set.shape)

print("Training features dataset 1 shape: ", x_train_1.shape)
print("Training labels dataset 1 shape: ", y_train_1.shape)
print("Training features dataset 2 shape: ", x_train_2.shape)
print("Training labels dataset 2 shape: ", y_train_2.shape)
print("Training features dataset 3 shape: ", x_train_3.shape)
print("Training labels dataset 3 shape: ", y_train_3.shape)
print("Training features dataset 4 shape: ", x_train_4.shape)
print("Training labels dataset 4 shape: ", y_train_4.shape)


Testing features dataset shape:  (197,)
Testing labels dataset shape:  (197,)
Training features dataset 1 shape:  (196,)
Training labels dataset 1 shape:  (196,)
Training features dataset 2 shape:  (197,)
Training labels dataset 2 shape:  (197,)
Training features dataset 3 shape:  (197,)
Training labels dataset 3 shape:  (197,)
Training features dataset 4 shape:  (197,)
Training labels dataset 4 shape:  (197,)


### Trying to train this model with some kind of cross validation method... hopefully 

In [54]:
model_name = "Category Ongeval(pototype 2).sav"
svm = LinearSVC(penalty="l1", loss='squared_hinge',dual=False )
SVM_model = CalibratedClassifierCV(svm)

train_list = [
    [x_train_1, y_train_1],
    [x_train_2, y_train_2],
    [x_train_3, y_train_3],
    [x_train_4, y_train_4]
]

size = len(train_list)

for index in range(0, size):
    train_features = None
    train_labels = None
    for i in range(0, size):
        if(index != i):
            # transformed = feature_vectorizer.transform(train_list[i][0])
            # print("Transformed Labels at Step-{} : {}".format(index + 1, i + 1))
            # SVM_model.fit(transformed, train_list[i][1])
            if(type(train_labels) == type(train_list[index])):
                train_features.append(train_list[i][0], ignore_index=True)
                train_labels.append(train_list[i][1], ignore_index=True)
            else:
                train_features = train_list[i][0]
                train_labels = train_list[i][1]

    transformed_features = feature_vectorizer.transform(train_features)
    # transformed_labels = feature_vectorizer.transform(train_labels)
    print("Transformed Labels at Step-{}".format(index + 1))

    SVM_model.fit(transformed_features, train_labels)
print("SVM_P score (test):",SVM_model.score(x_test_transformed, y_test_set))

Transformed Labels at Step-1
Transformed Labels at Step-2
Transformed Labels at Step-3
Transformed Labels at Step-4
SVM_P score (test): 0.5482233502538071


In [53]:
model_name = "Category Ongeval(pototype 2).sav"
# svm = LinearSVC(penalty="l1", loss='squared_hinge',dual=False )
# SVM_model = CalibratedClassifierCV(svm)
xgb_model=xgb.XGBClassifier(random_state=4, learning_rate= 0.5, max_depth= 3,
                            min_child_weight= 1, 
                            n_estimators= 100)

train_list = [
    [x_train_1, y_train_1],
    [x_train_2, y_train_2],
    [x_train_3, y_train_3],
    [x_train_4, y_train_4]
]

size = len(train_list)

for index in range(0, size):
    train_features = None
    train_labels = None
    for i in range(0, size):
        if(index != i):
            # transformed = feature_vectorizer.transform(train_list[i][0])
            # print("Transformed Labels at Step-{} : {}".format(index + 1, i + 1))
            # SVM_model.fit(transformed, train_list[i][1])
            if(type(train_labels) == type(train_list[index])):
                train_features.append(train_list[i][0], ignore_index=True)
                train_labels.append(train_list[i][1], ignore_index=True)
            else:
                train_features = train_list[i][0]
                train_labels = train_list[i][1]

    transformed_features = feature_vectorizer.transform(train_features)
    # transformed_labels = feature_vectorizer.transform(train_labels)
    print("Transformed Labels at Step-{}".format(index + 1))

    xgb_model.fit(transformed_features, train_labels)
print("XG Boost score (test):",xgb_model.score(x_test_transformed, y_test_set))

Transformed Labels at Step-1
Transformed Labels at Step-2
Transformed Labels at Step-3
Transformed Labels at Step-4
XG Boost score (test): 0.5177664974619289


In [58]:
from sklearn.model_selection import cross_val_score
svm = LinearSVC(penalty="l1", loss='squared_hinge',dual=False )
SVM_model_test = CalibratedClassifierCV(svm)
transformed_features = feature_vectorizer.transform(features)
cross_val_score(SVM_model_test, transformed_features, labels, cv=5)

array([0.59898477, 0.67005076, 0.6751269 , 0.6751269 , 0.62755102])

In [60]:
from sklearn.model_selection import cross_val_score
xgb_model_test=xgb.XGBClassifier(random_state=4, learning_rate= 0.5, max_depth= 3,
                            min_child_weight= 1, 
                            n_estimators= 100)
transformed_features = feature_vectorizer.transform(features)
cross_val_score(xgb_model_test, transformed_features, labels, cv=5)

array([0.50761421, 0.61928934, 0.70558376, 0.67005076, 0.60204082])

In [28]:
test = None
print(type(type(test)))

<class 'type'>


## ============================================================================================================

In [20]:
text_val = df_ly["Incident_Description"].astype(str)
text_val=text_val.to_numpy()
label_val = df_ly["Main_Categories"]
#val_tfidf = tfidf_vectorizer.fit(text_val)
tfidf_text_val= train_tfidf.transform(text_val)

In [21]:
classes = list(np.unique(df_ly.Main_Categories))
prob = SVM_cl_p.predict(tfidf_text_val) #Get predictions
prob_score = SVM_cl_p.predict_proba(tfidf_text_val) #Get prediction probsblity scores
print(type(prob))
print(prob.size)
print(type(prob_score))
print(prob_score.size)

<class 'numpy.ndarray'>
1892
<class 'numpy.ndarray'>
13244


In [50]:
test_prob_score = SVM_cl_p.predict_proba(tfidf_text_val[2])
print(test_prob_score.size)

7


In [39]:
for index in range(0, prob.size):
    print("This is prob: {} in index: {}".format(prob[index], index))
    for i in range(0, 7):
        print("     This is prob score: {} at index: {}".format(prob_score[i],i))
    if(index == 10):
        break

This is prob: Machines en handgereedschap in index: 0
     This is prob score: [0.12552724 0.12453582 0.14513011 0.12433426 0.12608752 0.28525015
 0.0691349 ] at index: 0
     This is prob score: [0.11851994 0.11596313 0.18978495 0.11519255 0.11681691 0.23350319
 0.11021933] at index: 1
     This is prob score: [0.13032124 0.1299462  0.14549248 0.1290827  0.13090292 0.09831778
 0.23593667] at index: 2
     This is prob score: [0.13122626 0.1333136  0.17180533 0.12997912 0.13186472 0.1740851
 0.12772586] at index: 3
     This is prob score: [0.12095807 0.12057714 0.16199772 0.11980851 0.12149796 0.09948358
 0.25567703] at index: 4
     This is prob score: [0.11716807 0.11679755 0.13364085 0.11605454 0.11769104 0.10775619
 0.29089177] at index: 5
     This is prob score: [0.11802951 0.11768984 0.26197413 0.11690779 0.11855632 0.12883363
 0.13800878] at index: 6
This is prob: Machines en handgereedschap in index: 1
     This is prob score: [0.12552724 0.12453582 0.14513011 0.12433426 0.12

In [22]:
import time
results = {"text_1":[],"label_1":[], "probability_score":[], "correct": []}
pred_percentage=0
pred_count=0
score_count = 7
for i in range (0,prob.size):
    scores = []
    results["text_1"].append(df_ly["Incident_Description"][i])
    results["label_1"].append(prob[i])
    results["probability_score"].append(prob_score[i])

    if prob[i]==df_ly["Main_Categories"][i]:
        pred_count+=1
        results["correct"].append(True)
    else:
        results["correct"].append(False)

    p = ((i + 1) / prob.size) * 100
    print ("Creating Predict File: {}%".format(round(p,2)), end="\r")

result_filename = "result_test (2).csv"
Results = pd.DataFrame(results)

Results.to_csv(result_filename, index=False)
print("\nResults were saved in file: ", result_filename)
pred_percentage= pred_count/prob.size
print("Prediction Percentage: ", pred_percentage)


Creating Predict File: 100.0%
Results were saved in file:  result_test (2).csv
Prediction Percentage:  0.6014799154334038


In [23]:
report = classification_report(df_ly["Main_Categories"], prob)
print(report)
print(type(report))

precision    recall  f1-score   support

                                 Aanrijding       0.00      0.00      0.00       154
Agressie en extreme lichamelijke inspanning       1.00      0.07      0.13        29
   Andere vormen van contact met voorwerpen       0.40      0.22      0.29       271
                       BHV inzet op locatie       0.00      0.00      0.00        97
    Gevaarlijke stoffen, brand en explosies       0.00      0.00      0.00        75
                Machines en handgereedschap       0.52      0.34      0.41       193
        Vallen (van personen of voorwerpen)       0.63      0.94      0.75      1073

                                   accuracy                           0.60      1892
                                  macro avg       0.36      0.22      0.23      1892
                               weighted avg       0.48      0.60      0.51      1892

<class 'str'>


In [ ]:
#Real World Use Case example

In [16]:
import time
import easygui

In [17]:
def preprocessing(data):
    print("\n******************** Data Preprocessing ******************\n")

    column_name = ""
    column_exists = False

    while(column_name == "" or column_exists == False):
        column_name = input("Please indicate what column contains the Incident Reports: ")
        print("Column Name: ", column_name)
        for col in data.columns:
            if(str(column_name) == str(col)):
                column_exists = True
                break

        if(column_exists == False):
            print("The column name you provided does not exist!")

    nltk.download('stopwords')

    stopword_list = set(nltk.corpus.stopwords.words('dutch'))

    #add words that aren't in the NLTK stopwords list
    added_stopwords = ["b", "u", "e", "ste", "kv", "aa", "a9", "mtr", '+', '1e', '2e', '<', '>', 'a', 'a1', 'a12', 'a2', 'a9',"één", "gis", "app", "h", 'o.a.', 'oa', 'en/of', "uur"
    "km", "mm", "v", "kg", "cm", "mm²", "mm2", "0ngever", "e", "de", "m"]
    updated_stopword_list = stopword_list.union(added_stopwords)

    # Dutch spacy tokens 
    # nlp = nl_core_news_sm.load()

    print("Loading Vectorizer...")
    

    print("Data Preprocessing...")
    text = data[column_name].astype(str)
    text = text.to_numpy()
    # text_tfidf = tfidf_vectorizer.fit(text)
    tfidf_text_val = train_tfidf.transform(text)

    return tfidf_text_val, column_name


In [28]:
def get_predictions(model, text, data, column):
    print("\n******************** Getting Predictions ******************\n")
    prob = model.predict(text) #Get predictions
    prob_score = model.predict_proba(text) #Get prediction probsblity scores
    prob_results = {"Incident_ID": [],"Incident_Report":[],"Model_Classification":[], "Probability_Scores": []}
    output_filename = ""

    print("*Output will automatically be an Excel file (.xlsx)")
    
    while(output_filename == ""):
        output_filename = input("Please enter output file name: ")

    output_filename = output_filename + ".xlsx"
    
    for i in range (0,prob.size):
        scores = []
        prob_results["Incident_ID"].append(data["Nr."][i])
        prob_results["Incident_Report"].append(data[column][i])
        prob_results["Model_Classification"].append(prob[i])
        prob_results["Probability_Scores"].append(prob_score[i])

        p = ((i + 1) / prob.size) * 100
        print ("Creating Predict File: {}%".format(round(p,2)), end="\r")

    print("\n\nSaving Predict File...")
    Prob_Results = pd.DataFrame(prob_results)

    print("\n\nCreating Excel File...")
    Prob_Results.to_excel(output_filename, index=False)
    print("\nResults were saved in file: ", output_filename)

In [29]:
def main():
    print("***************************************************************")
    print("Welcome to the Indicent Report Classification System!")
    print("***************************************************************\n")

    try:
        print("Please select file (.xlsx or .csv): ")
        filename = easygui.fileopenbox()
    except ValueError:
        print("\nPlease enter a valid file!")
        print("Please try again!")
    except:
        print("Something went wrong, please ty again or conact... Somebody...")

    model_name = "Category Ongeval(pototype).sav"

    # Dutch spacy tokens 
    nlp = nl_core_news_sm.load()

    data = pd.read_excel(filename)

    model = joblib.load(model_name)

    text, column_name = preprocessing(data)

    get_predictions(model, text, data, column_name)


main()

***************************************************************
Welcome to the Indicent Report Classification System!
***************************************************************

Please select file (.xlsx or .csv): 

******************** Data Preprocessing ******************

Column Name:  Beschrijving incident
Loading Vectorizer...
Data Preprocessing...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\micks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

******************** Getting Predictions ******************

*Output will automatically be an Excel file (.xlsx)
Creating Predict File: 100.0%

Saving Predict File...


Creating Excel File...

Results were saved in file:  OutPut.xlsx


In [103]:
!pip install openpyxl

Using legacy setup.py install for openpyxl, since package 'wheel' is not installed.
Using legacy setup.py install for et-xmlfile, since package 'wheel' is not installed.
    Running setup.py install for et-xmlfile: started
    Running setup.py install for et-xmlfile: finished with status 'done'
    Running setup.py install for openpyxl: started
    Running setup.py install for openpyxl: finished with status 'done'
